In [28]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pickle
import csv

from gpu import set_gpu

import tensorflow as tf
from latent.ml.dl_subclass import MLP, MLPbeta, CNN, MLPprop, get_train
from latent.ml.dl_manual import nn_pass, conv
import latent.session_subclass as session
import latent.utils.data_utils as prd
from latent.ml.lda import train_lda, eval_lda

set_gpu()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Num GPUs Available:  1


In [3]:
data, params = prd.load_caps_train('traindata.mat')

In [4]:
ep = 30
n_dof = np.max(params[:,0])
    
# Train NNs
mlp = MLP(n_class=n_dof)
mlp_beta = MLPbeta(n_class=n_dof)
mlp_prop = MLPprop(n_class=n_dof)
cnn = CNN(n_class=n_dof)

optimizer = tf.keras.optimizers.Adam()
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')
train_prop_accuracy = tf.keras.metrics.MeanSquaredError(name='train_prop_accuracy')

# Train neural networks
models = [mlp, mlp_beta, cnn]
models = [cnn]

trainmlp, traincnn, y_train, x_train_mlp, x_train_cnn, x_train_lda, y_train_lda, x_train_aug, emg_scale, scaler, x_min, x_max, prop = prd.prep_train_caps(data, params)

for model in models:
    if isinstance(model,CNN):
        ds = traincnn
    else:
        ds = trainmlp
 
    prop_b = isinstance(model, MLPprop)
 
    train_mod = get_train(prop = prop_b)

    for epoch in range(ep):
        # Reset the metrics at the start of the next epoch
        train_loss.reset_states()
        train_accuracy.reset_states()

        for x, y, y2 in ds:
            if prop_b:
                train_mod(x, y, model, optimizer, train_loss, train_accuracy, train_prop_accuracy, y2)
            else:
                train_mod(x, y, model, optimizer, train_loss, train_accuracy)

        if epoch == 0 or epoch == ep-1:
            print(
                f'Epoch {epoch + 1}, '
                f'Loss: {train_loss.result():.2f}, '
                f'Accuracy: {train_accuracy.result() * 100:.2f}, '
                f'MSE: {train_prop_accuracy.result():.2f} '
            )
            
    del train_mod

# Train aligned LDA
# y_train_aug = np.argmax(y_train, axis=1)[...,np.newaxis]

# mlp_enc = mlp.get_layer(name='enc')
# w_mlp, c_mlp,_, _, _ = train_lda(mlp_enc(x_train_mlp).numpy(),y_train_aug)

# mlpbeta_enc = mlp_beta.get_layer(name='enc')
# w_mlpbeta, c_mlpbeta,_, _, _ = train_lda(mlpbeta_enc(x_train_mlp).numpy(),y_train_aug)

# cnn_enc = cnn.get_layer(name='enc')
# temp = cnn_enc(x_train_cnn[:x_train_cnn.shape[0]//2,...]).numpy()
# temp2 = np.vstack((temp,cnn_enc(x_train_cnn[x_train_cnn.shape[0]//2:,...]).numpy()))
# w_cnn, c_cnn,_, _, _ = train_lda(temp2,y_train_aug)

# # Train LDA
# w,c, _, _, _ = train_lda(x_train_lda,y_train_lda)
# w_aug,c_aug, _, _, _ = train_lda(x_train_aug,y_train_aug)

mlp_w = mlp.get_weights()
mlpb_w = mlp_beta.get_weights()
cnn_w = cnn.get_weights()
mlpp_w = mlp_prop.get_weights()

Epoch 1, Loss: 1.58, Accuracy: 42.12, MSE: 0.00 
Epoch 30, Loss: 0.07, Accuracy: 97.69, MSE: 0.00 


In [5]:
enc = cnn.get_layer(name='enc')

In [37]:
test = enc.conv2(enc.conv1(x_train_cnn[[0],...]))
conv_w = enc.conv1.get_weights()
conv2_w = enc.conv2.get_weights()

In [40]:
conv2_w[0].shape

(3, 2, 1, 32)

In [18]:
np.squeeze(test[:,:,:,0])

array([[0.20118496, 0.17590678, 0.18570808, 0.1550428 ],
       [0.        , 0.00675926, 0.08845421, 0.        ],
       [0.04442876, 0.11047111, 0.13063185, 0.01857545],
       [0.        , 0.08245625, 0.09344479, 0.00084559],
       [0.15121415, 0.21976984, 0.17214307, 0.06722967],
       [0.16453835, 0.21301997, 0.22192857, 0.00274066]], dtype=float32)

In [30]:
np.squeeze(out)

array([[[ 0.20118497,  0.17590676,  0.18570807,  0.1550428 ],
        [-0.        ,  0.00675927,  0.0884542 , -0.        ],
        [ 0.04442876,  0.11047111,  0.13063185,  0.01857545],
        [-0.        ,  0.08245626,  0.09344478,  0.00084559],
        [ 0.15121414,  0.21976985,  0.17214307,  0.06722967],
        [ 0.16453836,  0.21301996,  0.22192857,  0.00274065]],

       [[ 0.018428  ,  0.03588767,  0.11781164,  0.11055478],
        [ 0.02309402,  0.09968477,  0.11537892,  0.0110324 ],
        [ 0.02093847,  0.11066228,  0.1263625 ,  0.01611886],
        [ 0.01968538,  0.12313664,  0.11057556, -0.        ],
        [ 0.01744111,  0.09784672,  0.06566927,  0.00204259],
        [ 0.01696988,  0.09803   ,  0.1371545 ,  0.02060483]]])

In [25]:
x = x_train_cnn[:2,...]

In [20]:
np.squeeze(padded)

array([[0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.68993932, 0.6364814 , 0.67741936, 0.78145695],
       [0.        , 0.04369863, 0.02972985, 0.41129032, 0.52317882],
       [0.        , 0.13506357, 0.10509808, 0.52419353, 0.57615894],
       [0.        , 0.05266867, 0.03941892, 0.51612902, 0.54304636],
       [0.        , 0.65596378, 0.6106711 , 0.78225809, 0.87417221],
       [0.        , 0.67590499, 0.68101662, 0.83870965, 0.84105963],
       [0.        , 0.        , 0.        , 0.        , 0.        ]])

In [26]:
padded = np.zeros((x.shape[0],x.shape[1]+2,x.shape[2]+1,x.shape[3]))
padded[:,1:-1,:-1,:] = x
out = np.zeros(x.shape)

for i in range(padded.shape[1]-2):
    for j in range(padded.shape[2]-1):
        temp = padded[:,i:i+3,j:j+2,:]
        out[:,i,j,:] = np.sum(np.sum(temp*conv_w[0][...,0],axis=1),axis=1) + conv_w[1][0]


In [36]:
out = conv(x,conv_w,stride=1)

In [6]:
# Save neural network architectures into csv
arch = ['RELU1','BN1','RELU2','BN2','RELU3','BN3','DEN','BN4','SOFTMAX','PROP']
w = {}
w_in = mlpp_w
folder = 'weights_p'
path = 'python/'

i = 0
for l in arch:
    w_layer = []
    if 'BN' in l:
        w_layer = np.vstack((np.vstack((np.vstack((w_in[i],w_in[i+1])),w_in[i+2])),w_in[i+3]))
        i += 4
    else:
        w_layer = np.vstack((w_in[i],w_in[i+1]))
        i += 2
    
    # Save to current folder
    with open(path + folder + '/' + l + '.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        # write multiple rows
        writer.writerows(w_layer)

    # Save to embedded
    try:
        with open('//192.168.20.2/config/modes/caps/upper_limb/dl_ctrl/DATA/' + folder + '/'  + l + '.csv', 'w', newline='') as f:
            writer = csv.writer(f)
            # write multiple rows
            writer.writerows(w_layer)
    except:
        print('no embedded')
    w[l] = w_layer

fill = len(emg_scale) - len(x_min)
temp = np.hstack((np.vstack((x_min[...,np.newaxis],np.zeros((fill,1)))), np.vstack((x_max[...,np.newaxis],np.zeros((fill,1))))))
scales = np.hstack((emg_scale,temp))

# Save to current folder
with open(path + folder + '/ARCH.csv', 'w', newline='') as f:
        writer = csv.writer(f,delimiter=',')
        # write multiple rows
        for i in arch:
            writer.writerow([i])

with open(path + folder + '/scales.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        # write multiple rows
        writer.writerows(scales)

# Save to embedded
try:
    with open('//192.168.20.2/config/modes/caps/upper_limb/dl_ctrl/DATA/' + folder + '/ARCH.csv', 'w', newline='') as f:
            writer = csv.writer(f,delimiter=',')
            # write multiple rows
            for i in arch:
                writer.writerow([i])

    with open('//192.168.20.2/config/modes/caps/upper_limb/dl_ctrl/DATA/' + folder + '/scales.csv', 'w', newline='') as f:
            writer = csv.writer(f)
            # write multiple rows
            writer.writerows(scales)
except:
    print('no embedded')

no embedded
no embedded
no embedded
no embedded
no embedded
no embedded
no embedded
no embedded
no embedded
no embedded
no embedded
